In [1]:
# Testing loading data from mongo db into clean dataframes
    # Mongo db -> Pandas dataframe -> csv

In [2]:
# Imports
import pandas as pd
import numpy as np
import json

In [3]:
# Pymongo import for connection to local client DB
import pymongo
from pymongo import MongoClient

# Mongo database and collection specification:
mongo_client = MongoClient('mongodb://localhost:27017/')
db = mongo_client.btcusd_db # Database specification
input_data = db.btcusd_collection #Collection specification (in database)
data = pd.DataFrame(list(input_data.find()))

In [4]:
# Verify that data was loaded from mongo into dataframe
data.head(10)

,_id,asks,bids,changes,channels,maker_order_id,message,price,product_id,reason,sequence,side,size,taker_order_id,time,trade_id,type
0,5ad526dce3ae7141c0170d17,"[[8014, 0.00646934], [8014.34, 0.001], [8015, ...","[[8013.99, 12.48282445], [8011, 23.9], [8010.9...",NaN,NaN,NaN,NaN,NaN,BTC-USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,snapshot
1,5ad526dce3ae7141c0170d18,NaN,NaN,NaN,NaN,ec5c5f7b-d32b-43b7-b946-3adca972e022,NaN,8014.00000000,BTC-USD,NaN,5.692402e+09,sell,0.00025066,71c294c5-700d-47e9-b578-bedb29fddcd5,2018-04-16T22:42:37.892000Z,41762773.0,last_match
2,5ad526dce3ae7141c0170d19,NaN,NaN,NaN,"[{'name': 'level2', 'product_ids': ['BTC-USD']...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,subscriptions
3,5ad526dce3ae7141c0170d1a,NaN,NaN,"[[buy, 7887.88000000, 0.0015]]",NaN,NaN,NaN,NaN,BTC-USD,NaN,NaN,NaN,NaN,NaN,2018-04-16T22:42:38.806Z,NaN,l2update
4,5ad526dce3ae7141c0170d1b,NaN,NaN,"[[sell, 8106.16000000, 0]]",NaN,NaN,NaN,NaN,BTC-USD,NaN,NaN,NaN,NaN,NaN,2018-04-16T22:42:38.807Z,NaN,l2update
5,5ad526dce3ae7141c0170d1c,NaN,NaN,"[[buy, 8010.99000000, 3.5]]",NaN,NaN,NaN,NaN,BTC-USD,NaN,NaN,NaN,NaN,NaN,2018-04-16T22:42:38.809Z,NaN,l2update
6,5ad526dce3ae7141c0170d1d,NaN,NaN,NaN,NaN,NaN,Failed to unsubscribe,NaN,NaN,You need to specify at least one product ID fo...,NaN,NaN,NaN,NaN,NaN,NaN,error
7,5ad526dce3ae7141c0170d1e,NaN,NaN,"[[sell, 8091.31000000, 19]]",NaN,NaN,NaN,NaN,BTC-USD,NaN,NaN,NaN,NaN,NaN,2018-04-16T22:42:38.817Z,NaN,l2update
8,5ad526dce3ae7141c0170d1f,NaN,NaN,"[[buy, 7931.07000000, 19]]",NaN,NaN,NaN,NaN,BTC-USD,NaN,NaN,NaN,NaN,NaN,2018-04-16T22:42:38.823Z,NaN,l2update
9,5ad526dce3ae7141c0170d20,NaN,NaN,"[[sell, 8059.26000000, 19]]",NaN,NaN,NaN,NaN,BTC-USD,NaN,NaN,NaN,NaN,NaN,2018-04-16T22:42:38.826Z,NaN,l2update


In [5]:
# Show information about test dataset from mongodb -> dataframe
data.info(verbose=True,  memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11713 entries, 0 to 11712
Data columns (total 17 columns):
_id               11713 non-null object
asks              1 non-null object
bids              1 non-null object
changes           11650 non-null object
channels          1 non-null object
maker_order_id    60 non-null object
message           1 non-null object
price             60 non-null object
product_id        11711 non-null object
reason            1 non-null object
sequence          60 non-null float64
side              60 non-null object
size              60 non-null object
taker_order_id    60 non-null object
time              11710 non-null object
trade_id          60 non-null float64
type              11713 non-null object
dtypes: float64(2), object(15)
memory usage: 1.5+ MB


In [6]:
# Display unique response types from 'type' column
data['type'].unique()

array(['snapshot', 'last_match', 'subscriptions', 'l2update', 'error',
       'match'], dtype=object)

In [7]:
# Create individual dataframes for response types
    # Main relevant response types: snapshot, l2update, match, last_match
snapshot = pd.DataFrame(list(input_data.find({'type':'snapshot'})))
l2update = pd.DataFrame(list(input_data.find({'type':'l2update'})))
match = pd.DataFrame(list(input_data.find({'type':'match'})))

last_match = pd.DataFrame(list(input_data.find({'type':'last_match'})))
subscriptions = pd.DataFrame(list(input_data.find({'type':'subscriptions'})))
error = pd.DataFrame(list(input_data.find({'type':'error'})))
    # Error for unsubscribe message is issue with GDAX python API websocket call

In [8]:
snapshot.head()

,_id,asks,bids,product_id,type
0,5ad526dce3ae7141c0170d17,"[[8014, 0.00646934], [8014.34, 0.001], [8015, ...","[[8013.99, 12.48282445], [8011, 23.9], [8010.9...",BTC-USD,snapshot


In [9]:
l2update.head()

,_id,changes,product_id,time,type
0,5ad526dce3ae7141c0170d1a,"[[buy, 7887.88000000, 0.0015]]",BTC-USD,2018-04-16T22:42:38.806Z,l2update
1,5ad526dce3ae7141c0170d1b,"[[sell, 8106.16000000, 0]]",BTC-USD,2018-04-16T22:42:38.807Z,l2update
2,5ad526dce3ae7141c0170d1c,"[[buy, 8010.99000000, 3.5]]",BTC-USD,2018-04-16T22:42:38.809Z,l2update
3,5ad526dce3ae7141c0170d1e,"[[sell, 8091.31000000, 19]]",BTC-USD,2018-04-16T22:42:38.817Z,l2update
4,5ad526dce3ae7141c0170d1f,"[[buy, 7931.07000000, 19]]",BTC-USD,2018-04-16T22:42:38.823Z,l2update


In [10]:
match.head()

,_id,maker_order_id,price,product_id,sequence,side,size,taker_order_id,time,trade_id,type
0,5ad526e8e3ae7141c0171130,ec5c5f7b-d32b-43b7-b946-3adca972e022,8014.00000000,BTC-USD,5692403770,sell,0.00074934,5281588f-b333-4754-b0f3-2eb90ba1bf14,2018-04-16T22:42:51.576000Z,41762774,match
1,5ad526e8e3ae7141c0171132,8ffdf046-c9a0-4b9a-9828-4717ef098440,8014.00000000,BTC-USD,5692403772,sell,0.00378000,5281588f-b333-4754-b0f3-2eb90ba1bf14,2018-04-16T22:42:51.576000Z,41762775,match
2,5ad526e8e3ae7141c0171134,bef3d02b-9d15-4cb0-b9f2-843ea1c7a320,8014.00000000,BTC-USD,5692403774,sell,0.03947066,5281588f-b333-4754-b0f3-2eb90ba1bf14,2018-04-16T22:42:51.576000Z,41762776,match
3,5ad526f2e3ae7141c0171767,972ef644-9fac-4bc5-b505-e38be60a8372,8013.99000000,BTC-USD,5692406161,buy,0.00110000,706b1b43-f993-443b-ace7-6677f7d8cb74,2018-04-16T22:43:01.143000Z,41762777,match
4,5ad526f4e3ae7141c0171806,6189782c-a498-4cb0-b39a-a23f9dfa4289,8014.00000000,BTC-USD,5692406400,sell,0.00189000,314ff478-7507-400f-b088-769e87381a4d,2018-04-16T22:43:03.041000Z,41762778,match


In [11]:
last_match.head()

,_id,maker_order_id,price,product_id,sequence,side,size,taker_order_id,time,trade_id,type
0,5ad526dce3ae7141c0170d18,ec5c5f7b-d32b-43b7-b946-3adca972e022,8014.00000000,BTC-USD,5692402034,sell,0.00025066,71c294c5-700d-47e9-b578-bedb29fddcd5,2018-04-16T22:42:37.892000Z,41762773,last_match


In [12]:
subscriptions['channels'][0]

[{'name': 'level2', 'product_ids': ['BTC-USD']},
 {'name': 'matches', 'product_ids': ['BTC-USD']}]

In [13]:
# Format snapshot dataframe (snapshot is initial snapshot of L2 orderbook state)
snapshot.head()

,_id,asks,bids,product_id,type
0,5ad526dce3ae7141c0170d17,"[[8014, 0.00646934], [8014.34, 0.001], [8015, ...","[[8013.99, 12.48282445], [8011, 23.9], [8010.9...",BTC-USD,snapshot


In [14]:
# Extract asks/bid individual column of array of arrays into lists
snapshot_asks = snapshot[['asks'][0]][0]
snapshot_bids = snapshot[['bids'][0]][0]

# Convert list (of array of arrays) into dataframe
snapshot_asks_df =pd.DataFrame(snapshot_asks)
snapshot_bids_df =pd.DataFrame(snapshot_bids)
   
# Rename columns to snapshot array format:
    # snapshot array format: [price, size]
    # Ask = sell price, bid = buy price
snapshot_asks_df.rename(columns ={0:'price',1:'size'}, inplace =True)
snapshot_bids_df.rename(columns ={0:'price',1:'size'}, inplace =True)
snapshot_asks_df['side'] = "sell"
snapshot_bids_df['side'] = "buy"

In [15]:
snapshot_asks_df.head()

,price,size,side
0,8014,0.00646934,sell
1,8014.34,0.001,sell
2,8015,0.021821,sell
3,8015.15,0.006,sell
4,8015.19,0.02384074,sell


In [16]:
snapshot_bids_df.head()

,price,size,side
0,8013.99,12.48282445,buy
1,8011,23.9,buy
2,8010.99,3.501,buy
3,8010,1,buy
4,8008,0.001,buy


In [17]:
l2update = l2update[['_id','changes','time']]
l2update.head()

,_id,changes,time
0,5ad526dce3ae7141c0170d1a,"[[buy, 7887.88000000, 0.0015]]",2018-04-16T22:42:38.806Z
1,5ad526dce3ae7141c0170d1b,"[[sell, 8106.16000000, 0]]",2018-04-16T22:42:38.807Z
2,5ad526dce3ae7141c0170d1c,"[[buy, 8010.99000000, 3.5]]",2018-04-16T22:42:38.809Z
3,5ad526dce3ae7141c0170d1e,"[[sell, 8091.31000000, 19]]",2018-04-16T22:42:38.817Z
4,5ad526dce3ae7141c0170d1f,"[[buy, 7931.07000000, 19]]",2018-04-16T22:42:38.823Z


In [18]:
match.head()

,_id,maker_order_id,price,product_id,sequence,side,size,taker_order_id,time,trade_id,type
0,5ad526e8e3ae7141c0171130,ec5c5f7b-d32b-43b7-b946-3adca972e022,8014.00000000,BTC-USD,5692403770,sell,0.00074934,5281588f-b333-4754-b0f3-2eb90ba1bf14,2018-04-16T22:42:51.576000Z,41762774,match
1,5ad526e8e3ae7141c0171132,8ffdf046-c9a0-4b9a-9828-4717ef098440,8014.00000000,BTC-USD,5692403772,sell,0.00378000,5281588f-b333-4754-b0f3-2eb90ba1bf14,2018-04-16T22:42:51.576000Z,41762775,match
2,5ad526e8e3ae7141c0171134,bef3d02b-9d15-4cb0-b9f2-843ea1c7a320,8014.00000000,BTC-USD,5692403774,sell,0.03947066,5281588f-b333-4754-b0f3-2eb90ba1bf14,2018-04-16T22:42:51.576000Z,41762776,match
3,5ad526f2e3ae7141c0171767,972ef644-9fac-4bc5-b505-e38be60a8372,8013.99000000,BTC-USD,5692406161,buy,0.00110000,706b1b43-f993-443b-ace7-6677f7d8cb74,2018-04-16T22:43:01.143000Z,41762777,match
4,5ad526f4e3ae7141c0171806,6189782c-a498-4cb0-b39a-a23f9dfa4289,8014.00000000,BTC-USD,5692406400,sell,0.00189000,314ff478-7507-400f-b088-769e87381a4d,2018-04-16T22:43:03.041000Z,41762778,match


In [19]:
# Restucture l2update to have [side,price,size] from 'changes' column in seperate columns for parsing:
l2update.head()

,_id,changes,time
0,5ad526dce3ae7141c0170d1a,"[[buy, 7887.88000000, 0.0015]]",2018-04-16T22:42:38.806Z
1,5ad526dce3ae7141c0170d1b,"[[sell, 8106.16000000, 0]]",2018-04-16T22:42:38.807Z
2,5ad526dce3ae7141c0170d1c,"[[buy, 8010.99000000, 3.5]]",2018-04-16T22:42:38.809Z
3,5ad526dce3ae7141c0170d1e,"[[sell, 8091.31000000, 19]]",2018-04-16T22:42:38.817Z
4,5ad526dce3ae7141c0170d1f,"[[buy, 7931.07000000, 19]]",2018-04-16T22:42:38.823Z


In [20]:
l2update['changes'][0]

[['buy', '7887.88000000', '0.0015']]

In [21]:
l2update['changes'][0][0]

['buy', '7887.88000000', '0.0015']

In [22]:
print(l2update['changes'][0][0][0])
print(l2update['changes'][0][0][1])
print(l2update['changes'][0][0][2])

buy
7887.88000000
0.0015


In [23]:
l2update_clean = l2update
l2update_clean['side'] = l2update['changes'][0][0][0]
l2update_clean['price'] = l2update['changes'][0][0][1]
l2update_clean['size'] = l2update['changes'][0][0][2]
l2update_clean = l2update_clean.drop(['changes'], axis=1)


In [24]:
l2update_clean.head()

,_id,time,side,price,size
0,5ad526dce3ae7141c0170d1a,2018-04-16T22:42:38.806Z,buy,7887.88000000,0.0015
1,5ad526dce3ae7141c0170d1b,2018-04-16T22:42:38.807Z,buy,7887.88000000,0.0015
2,5ad526dce3ae7141c0170d1c,2018-04-16T22:42:38.809Z,buy,7887.88000000,0.0015
3,5ad526dce3ae7141c0170d1e,2018-04-16T22:42:38.817Z,buy,7887.88000000,0.0015
4,5ad526dce3ae7141c0170d1f,2018-04-16T22:42:38.823Z,buy,7887.88000000,0.0015


In [26]:
# Save test data to .csv format
    #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html
data.to_csv("test_data/raw_mongo_input.csv",header = True,encoding='utf-8',index =False)
#l2update.to_csv("test_data/l2update.csv",header=True,encoding='utf-8',index =False)
l2update_clean.to_csv("test_data/l2update.csv",header=True,encoding='utf-8',index =False)
snapshot_asks_df.to_csv("test_data/snapshot_asks.csv",header=True,encoding='utf-8',index =False)
snapshot_bids_df.to_csv("test_data/snapshot_bids.csv",header=True,encoding='utf-8',index =False)

last_match.to_csv("test_data/last_match.csv",header=True,encoding='utf-8')
match.to_csv("test_data/match.csv",header=True,encoding='utf-8',index =False)
subscriptions.to_csv("test_data/subscriptions.csv",header=True,encoding='utf-8',index =False)